О содержании статьи.

Об интерпретации моделей машинного обучения в целом (кратко).

Локальное упрощение и оценка важности признаков в упрощенной модели (иллюстрация).

Введение в LIME и SHAP

Схема из кругов: интерпретация моделей со сложными признаками, с простыми признаками (переход - выбор конкретного примера, т. е. локальная интерпретация), LIME, SHAP, библиотека SHAP, saliency maps, mean impurity decrease. LIME - подход к упрощению модели, SHAP - подход к анализу вклада признаков в модели, в котором для вычислительной эффективности требуется простота модели. Интерпретация сложной модели = локальное упрощение модели + оценка важности признаков в простой модели. LIME - подход к упрощению модели, а SHAP отвечает на вопрос о том, как лучше всего оценивать важность признаков в простой модели. Получение модели с простыми признаками.

Локальное упрощение модели: терминология, обозначения и примеры

Алгоритм LIME

Shapley additive explanation: вклад признаков в модель
(+ интересное замечание: анализ не только моделей, но и самой целевой зависимости)

Linear LIME + kernel SHAP

SHAP на практике

https://arxiv.org/pdf/2002.11097.pdf

https://papers.nips.cc/paper/2020/file/0d770c496aa3da6d2c3f2bd19e7b9d6b-Paper.pdf

https://papers.nips.cc/paper/2020/file/32e54441e6382a7fbacbbbaf3c450059-Paper.pdf

-----

(Введение, о содержании статьи, ссылки на работы)

## Интерпретация моделей машинного обучения

Модели машинного обучения (такие как нейронные сети, машины опорных векторов, ансамбли решающих деревьев) являются "прозрачными" в том смысле, что все происходящие внутри них вычисления известны. Но тем не менее часто говорят, что модели машинного обучения плохо интерпретируемы. Здесь имеется в виду то, что процесс принятия решения не удается представить в понятной человеку форме, то есть:

1. Понять, какие признаки или свойства входных данных влияют на ответ
2. Разложить алгоритм принятия решения на понятные составные части
3. Объяснить смысл промежуточных результатов, если они есть
3. Описать в текстовом виде алгоритм принятия решения (возможно, с привлечением схем или графиков)

Достичь полной интерпретируемости в машинном обучении, как правило, не удается, но даже частичная интерпретация может существенно помочь. Если мы узнаем, на что именно обращает внимание алгоритм, какими правилами руководствуется, то сможем оценить правдоподобность этих правил и тем самым распознать переобучение. Также мы сможем лучше понять, почему модель ошибается на тех или иных примерах, что может помочь улучшить алгоритм обучения. Кроме того, интерпретировав алгоритм, мы можем открыть для себя что-то новое о свойствах исследуемых данных (например, какие признаки в табличных данных в наибольшей степени влияют на ответ).

> Humans usually have prior knowledge about the application domain, which they can use to accept (trust) or reject a prediction if they understand the reasoning behind it. ([Ribeiro et al., 2016]($Why Should I Trust You?: Explaining the Predictions of Any Classifier$))

Обзор способов интерпретации моделей машинного обучения можно найти, например, в [Linardatos et al., 2020]($Explainable AI: A Review of Machine Learning Interpretability Methods$) и [Li et al., 2021]($Interpretable Deep Learning: Interpretation, Interpretability, Trustworthiness, and Beyond$). 

### Интерпретация моделей для оценки качества

Авторы методa LIME ([Ribeiro et al., 2016]($Why Should I Trust You?: Explaining the Predictions of Any Classifier$)) делают акцент на ситуации, когда требуется оценить качество обученной модели (например, чтобы сделать выбор между одной из нескольких моделей). Оценка точности на тестовой выборке не всегда позволяет хорошо оценить качество из-за следующих проблем:

1. <p><b>Сдвиг данных</b> (data shift). Эта ситуация означает, что данные, на которых модель будет применяться, среднестатистически отличаются от тех, на которых модель обучалась и тестировалась, то есть распределение входных данных отличается при тестировании и применении: $P_{test}(X, Y) \neq P_{usage}(X, Y)$. Поскольку $P(X, Y) = P(Y|X)P(X)$, то двумя вариантами сдвига данных является сдвиг в исходных данных $P(X)$ и сдвиг целевой переменной $P(Y|X)$.</p>
<p>Одна из проблем сдвига данных в том, что большинство метрик качества (accuracy, MSE, logloss, F1 и другие) зависят от распределения исходных данных $P(X)$, то есть, упрощенно говоря, от соотношения разных типов примеров в датасете. Например, пусть модель тестировалась на датасете, в котором 80% изображений были высокого качества (HQ), а применяться будет в условиях, когда, наоборот, 80% изображений будут низкого качества (LQ). Пусть мы сравниваем две модели: на HQ-изображениях точность первой модели лучше, чем второй, а на LQ-изображениях, наоборот, точность второй модели лучше, чем первой. Если при тестировании большая часть изображений были HQ, то мы сделаем вывод, что первая модель лучше, тогда как на самом деле лучше была бы вторая.</p>  
<p>Другая проблема сдвига данных в том, что в обучающих и тестовых данных могут присутствовать такие корреляции, которые не обобщаются на другие выборки. Например, мы классифицируем животных по изображению, но большинство изображений рыб, которые у нас имеются, содержат также пальцы рыбака (эти изображения мы используем как при обучении, так и при тестировании). Модель может научиться классифицировать как рыбу изображение, содержащее пальцы, что в целом неверно и может не работать на других выборках (см. также <a href=$Inductive Biases for Deep Learning of Higher-Level Cognition$>Goyal and Bengio, 2020</a> и обзор <a href=$Априорные гипотезы и регуляризация в машинном обучении$>Априорные гипотезы и регуляризация в машинном обучении</a>; пример с рыбами см. в <a href=$Approximating CNNs with Bag-of-local-Features models works surprisingly well on ImageNet$>Brendel and Bethge, 2019</a>).</p>

2. **Утечка данных** (data leakage, или target leakage). Например, ID пациента может сильно коррелировать с диагнозом, но только в текущем датасете (который поделен на обучающую и тестовую часть). Модель, предсказывающая диагноз по ID, будет иметь высокую точность на тестовом датасете, но в целом очевидно, что в данной задаче такой способ предсказания некорректен и не будет хорошо работать на других данных. Утечка данных является частным случаем сдвига данных, поскольку зависимость ID $\to$ диагноз была в $P_{train}(X, Y)$ и $P_{test}(X, Y)$, но ее не будет в $P_{usage}(X, Y)$. Избавиться от утечки данных не всегда просто.

> Еxample for this is KDD-Cup 2008 breast cancer prediction competition, where the patient ID contained an obvious leak. It is by no means obvious that removing this feature would leave a leakage-free dataset, however. Assuming different ID ranges correspond to different health care facilities (in different geographical locations, with different equipment), there may be additional traces of this in the data. If for instance the imaging equipment's grey scale is slightly different and in particular grey levels are higher in the location with high cancer rate, the model without ID could pick up this leaking signal from the remaining data, and the performance estimate
would still be optimistic (the winners show evidence of this in their report). ([Kaufman et al., 2011]($Leakage in Data Mining: Formulation, Detection, and Avoidance$))

Если удастся интерпретировать модель (хотя бы приблизительно), то мы получим дополнительную информацию, которая поможет надежнее оценить ее качество в условиях возможной утечки и сдвига данных.

### Локальная интерпретация моделей

Вместо попыток интерпретировать модель целиком, что может быть очень сложно, мы можем рассмотреть задачу интерпретации ответа модели $f$ на конкретном, фиксированном примере $x_0$. Например, если на данном изображении модель распознала собаку, то почему она распознала собаку? Какие части и свойства изображения повлияли на предсказание модели?

Для ответа на этот вопрос мы можем изменять $x_0$ и смотреть, как изменится при этом ответ модели, то есть мы изучаем зависимость $f(x_0 + \Delta x)$ от $\Delta x$. При этом возможно удастся с хорошей точностью аппроксимировать эту зависимость простой функцией $g(\Delta x)$. Такой подход называется локальной аппроксимацией модели в окрестности точки $x_0$.

Например, рассчитав градиент $\nabla f(x)$ в точке $x_0$ мы узнаем, как изменится ответ при очень малых изменениях $\Delta x$. При этом мы получаем локальную линейную аппроксимацию (в курсе высшей математики такая аппроксимация называется дифференциалом функции $f$). Такой подход используется, например, при расчете так называемых saliency maps в компьютерном зрении ([Simonyan et al., 2013]($Deep Inside Convolutional Networks: Visualising Image Classification Models and Saliency Maps$)) - производных выходных значений сети по отдельным пикселям изображения. Но такая аппроксимация далеко не всегда адекватна:

1. Производная локальна и не говорит о том, как изменится ответ при существенных изменениях $\Delta x$. Например, если один из признаков достиг состояния "насыщения", то есть значение данного признака более чем достаточно, чтобы сделать какой-то вывод о целевой переменной, то производная по нему почти равна нулю. Эффект будет лишь если мы сильно изменим данный признак.
2. В некоторых моделях (решающих деревьях) производная либо равна нулю, либо не существует.
3. Для бинарных признаков производная не всегда информативна, поскольку малое изменение признака ведет в "невозможную" область нецелого значения, в котором модель и не обязана работать корректно.

Есть и другой подход к локальной интерпретации модели: в качестве $\Delta x$ мы можем рассматривать некий набор осмысленных, не бесконечно малых изменений входных данных. Именно такой подход лежит в основе метода интерпретации LIME, который мы рассмотрим далее.

## LIME: Local Interpretable Model-agnostic Explanations

LIME - это подход к интерпретации ответа модели $f(x_0)$ на конкретном тестовом примере $x_0$ с помощью вычисления значений $f(x_0 + \Delta x)$ для некоторого конечного набора значений $\Delta x$. Иллюстрация работы метода LIME приведена на *рис. 1*. Чтобы формально описать метод LIME, нам понадобится ввести ряд обозначений, которые мы будем использовать и в следующих частях обзора.

<img src="assets/lime2.jpg" width="600" align="center">

<center><i>Рис. 1. Иллюстрация работы метода LIME.</i></center>

Мы интерпретируем модель $f$ на примере $x_0$:

- $f: X \to Y$ - исходная модель
- $x_0 \in X$ - выбранный тестовый пример, предсказание на котором $f(x_0)$ интерпретируется

### Локальное упрощенное представление

Мы вводим $M$ осмысленных, интерпретируемых изменений примера $x_0$ - например, для изображений таким изменением может быть удаление отдельного суперпикселя, то есть участка изображения с похожим содержимым и четкими границами (*рис. 1b*). Каждое изменение бинарно, то есть оно либо есть, либо нет. Соответственно, мы получаем $2^M$ различных вариантов $\Delta x$. Наличие или отсутствие каждого из изменений можно описать числом 0 или 1: из этих чисел можно собрать бинарный вектор $z$ размерностью $M$, который будем называть *упрощенным представлением* (*рис. 1c*).

- $M$ - количество изменений
- $z_i \in \{0, 1\}$ - индикатор $i$-го изменения
- $z \in \{0, 1\}^M$ - вектор упрощенного представления
- $h: \{0, 1\}^M \to X$ - функция, преобразующая вектор упрощенного представления $z$ в $x_0 + \Delta x$

Например, на *рис. 1* функция $h$ работает следующим образом: все "пристствующие" суперпиксели ($z_i = 1$) рисуются без изменений. Все "отсутствующие" суперпиксели ($z_i = 0$) заполняются белым цветом (либо, как вариант, усредненным цветом соседних суперпикселей). При этом $h([1, 1, \dots, 1]) = x_0$, поскольку вектор из единиц означает отсутствие всех изменений. Фактически выбор функции $h$ равносилен выбору семантики упрощенного представления (то есть что означают и как влияют на пример отдельные элементы $z_i$).

Функцию $h$ мы можем выбрать произвольно, но так, чтобы отдельные элементы упрощенного представления $z$ были интерпретируемы. При этом мы надеемся, что объяснить предсказание модели на $x_0$ можно интерпретировать, изучая, как влияют на ответ изменения отдельных элементов вектора $z$. Конечно, функция $g$ может быть выбрана неудачно. Например, если модель определяет стиль фотографии как "ретро" при наличии оттенка "сепия", то изменения отдельных суперпикселей не помогут интерпретировать модель. Если функция $h$ была выбрана неудачно, то всегда можно попробовать заново с другой $h$, то есть с другими по смыслу изменениями $x_0$.

Далее мы обучаем простую, интерпретируемую модель $g$, которая аппроксимирует ответ $f$, используя $z$. Для этого нам потребуется получить ответ модели $f$ для разных $z$ (то есть для разных $x_0 + \Delta x$, поскольку $z$ определяет $\Delta x$) и таким образом собрать обучающую выборку для модели $g$.

### Объясняющая модель

Теперь мы можем обучить модель $g$ предсказывать значение $f(h(z))$ по вектору упрощенного представления $z$. При этом модель $g$ должна быть простой и интерпретируемой (поскольку смысл метода LIME в интерпретации). Например, это может быть линейная модель или решающее дерево. Чтобы обучить модель, нужно собрать обучающую выборку. Для этого нужно выбрать $N$ разных значений вектора $z$ и получить для них предсказание модели $f$.

- $g: \{0, 1\}^M \to Y$ - объясняющая модель
- $\big\{ z^{(i)}, f(h(z^{(i)})) \big\}_{i=1}^N$ - обучающая выборка для объясняющей модели

Максимально возможный размер обучающей выборки равен $2^M$, но обычно $M$ велико, и приходится ограничиться перебором лишь некоторых значений $z$. Авторы предлагают уделять основное внимание таким $z$, которые близки к вектору из единиц: это соответствует небольшим изменениям в $x_0$ (чем больше нулей в $z$, тем больше одновременных изменений $x_0$ мы рассматриваем). Введем функцию $\pi(z)$, определяющую меру близости $h(z)$ к $x_0$, и назначим веса $\pi(z^{(i)})$ примерам из обучающей выборки.

- $\pi: \{0, 1\}^M \to \mathbb{R}$ - мера близости $h(z)$ к $x_0$
- $\big\{ w^{(i)} = \pi(z^{(i)}) \big\}_{i=1}^N$ - веса примеров из обучающей выборки

*Примечание.* В kernel SHAP, который мы рассмотрим в следующих разделах, в качестве $\pi$ берется функция, назначающая большие веса как векторам с большим количеством единиц, так и векторам с большим количеством нулей. Так мы акцентируем внимание в том числе на значениях $z$ с большим количеством нулей, то есть на отдельных интерпретируемых компонентах (напрмер, отдельных суперпикселях в случае изображений).

Для обучения модели $g$ осталось выбрать функцию потерь - например, среднеквадратичное отклонение. Эта функция будет сравнивать предсказания моделей $f$ и $g$. Авторы предлагают также использовать "штраф за сложность" $\Omega(g)$ - например, количество ненулевых весов в линейной модели. Если в качестве функции потерь выбрано среднеквадратичное отклонение, то задача оптимизации формулируется следующим образом:

$\sum\limits_{i=1}^N w^{(i)} \Big( g(z^{(i)}) - f(h(z^{(i)})) \Big)^2 + \Omega(g) \to \min\limits_g$

В данной формуле мы считаем квадрат разности предсказаний объясняющей модели $g(z^{(i)})$ и исходной модели $f(h(z^{(i)}))$, и считаем взвешенную сумму по обучающей выборке, используя веса $w^{(i)}$. Кроме того мы прибавляем штраф за сложность объясняющей модели $\Omega(g)$.

### Резюме

Суть подхода LIME в том, что мы аппроксимируем предсказание модели $f$ в окрестности тестового примера $x_0$ более простой, легко интерпретируемой моделью $g$, которая использует упрощенное представление $z$. Например, если модель $g$ линейна, то каждому изменению $z_i$ (например, суперпикселю в изображении) сопоставляется некий вес. 

При этом мы надеемся, что такая аппроксимация адекватна, то есть наличие $i$-го изменения линейно влияет на предсказание модели $f$. В некоторых случаях это может оказаться совсем не так, и модель $g$ не сможет хорошо обучиться (функция потерь остенется высокой). Например, в случае изображений это может означать, что мы не можем линейно влиять на предсказание модели, удаляя отдельные суперпиксели. Возможно, модель ориентируется не на отдельные объекты, а на цвет изображения в целом. Тогда можно попробовать использовать другое упрощенное представление, элементами которого является информация об усредненном цвете изображения.

Работа алгоритма LIME не зависит от вида модели $f$ (нейронная сеть, решающие деревья и т. д.) и никак явно не использует информацию о том, как модель устроена "изнутри", то есть LIME является "model-agnostic" алгоритмом интерпретации.

### Примеры и обсуждение

На *рис. 2* мы видим объяснение предсказания сверточной нейронной сети Inception ([Szegedy et al., 2014]($Going Deeper with Convolutions$)). Сначала мы рассматриваем выходной нейрон, соответствующий классу "Electric guitar", и пытаемся аппроксимировать значение на этом нейроне с помощью линейной модели $g$, которая использует информацию о наличии или отстутствии суперпикселей ($M$ бинарных признаков, где $M$ - количество суперпикселей). В результате для каждого суперпикселя мы получаем вес, то есть вклад этого суперпикселя в предсказание "Electric guitar", и выделяем суперпиксели с наибольшим весом. Далее повторяем тот же алгоритм для двух других выходных нейронов, соответствующих классам "Acoustic guitar" и "Labrador". Как можно видеть из примера, алгоритм LIME концептуально достаточно прост.

<img src="assets/lime.jpg" width="800" align="center">

<center><i>Рис. 2. Пример результатов, полученных с помощью метода LIME. Отмечены суперпиксели, имеющие наибольшие веса в линейной объясняющей модели.</i></center>

## Введение в LIME и SHAP

В этом разделе мы начнем знакомство с методами LIME ([Ribeiro et al., 2016]($Why Should I Trust You?: Explaining the Predictions of Any Classifier$)) и SHAP ([Lundberg and Lee, 2017]($A Unified Approach to Interpreting Model Predictions$)). Сначала мы рассмотрим общие принципы работы этих методов и их взаимосвязь, введем необходимую терминологию и обозначения. В следующих разделах рассмотрим эти методы более подробно и формально.

Например, в случае изображений это может быть закрашивание однотонным цветом определенных участков, изменение цветов и т. д.; в случае табличных данных мы можем заменять один или несколько признаков на среднее или медианное значение по датасету. 

- Схема из кругов: интерпретация моделей со сложными признаками, с простыми признаками (переход - выбор конкретного примера, т. е. локальная интерпретация), LIME, SHAP, библиотека SHAP, saliency maps, mean impurity decrease.

- Интерпретация сложной модели = локальное упрощение модели + оценка важности признаков в простой модели. LIME - подход к упрощению модели, SHAP - подход к анализу вклада признаков в модели, в котором для вычислительной эффективности требуется простота модели.

- Локальное упрощение модели: терминология, обозначения и примеры